<a href="https://colab.research.google.com/github/ayushjain1144/NER/blob/master/NER_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import numpy as np
import sys

# Loading the features and vocabulary

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!ln -s content/gdrive/My\ Drive/NER /ner_dir

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
%cd ..
%cd /ner_dir


/
/content/gdrive/My Drive/NER


In [ ]:
!ls

activations.py		  loss.py		README.md
Conll.ipynb		  NER_NN.ipynb		test_features.npy
dataset			  NER_NN_network.ipynb	train_features.npy
initial_experiment.ipynb  nn.py			val_features.npy
initialization.py	  __pycache__		vocab.npy


In [ ]:
import activations
import loss
import initialization as init_layer

In [ ]:
vocab = np.load('vocab.npy')
train_features = np.load('train_features.npy')
test_features = np.load('test_features.npy')
val_features = np.load('val_features.npy')

In [ ]:
print(train_features.shape)
print(vocab.shape)
print(test_features.shape)
print(val_features.shape)

(204566, 900)
(26872, 300)
(46665, 900)
(51577, 900)


In [ ]:
train_features[:10]

array([[ 1.        ,  0.        ,  0.        , ...,  0.03063965,
        -0.10351562,  0.203125  ],
       [-0.92605459, -1.13792351, -0.7880129 , ..., -0.70885908,
        -1.25367734,  0.080868  ],
       [ 0.00982666,  0.2265625 ,  0.28125   , ..., -0.02233887,
         0.12695312, -0.04150391],
       ...,
       [-1.97499414,  0.00412577,  0.30153385, ...,  0.25416797,
        -0.06190594, -0.14426987],
       [-0.08007812,  0.15625   , -0.35351562, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.        ,  0.        ,  0.        , ...,  0.65237435,
        -0.67437781,  0.23527849]])

In [ ]:
C = 1
word_vector_dim = 300
num_tags = 10

# Neural Netwwork Architecture

In [2]:
nn_architecture = [
  {"layer_size": 900, "activation": "none"},
  {"layer_size": 300, "activation": "relu"},
  {"layer_size": 100, "activation": "relu"},
  {"layer_size":10, "activation": "softmax"}
]

In [4]:
nn_architecture[1]['activation']

'relu'

# Initialize Parameters

In [6]:
def initialize_parameters(nn_architecture, initialization = "range_initialization", seed=5):

  parameters = {}
  num_layers = len(nn_architecture)

  for i in range(1, num_layers):

    if initialization == "range_initialization":
      parameters['W' + str(i)] = init_layer.range_initializtion(nn_architecture[i]["layer_size"],
                                            nn_architecture[i - 1]["layer_size"], seed)
    else:
      parameters['W' + str(i)] = init_layer.random_initializtion(nn_architecture[i]["layer_size"],
                                            nn_architecture[i - 1]["layer_size"], seed)

    parameters['b' + str(i)] = np.zeros((nn_architecture[i]["layer_size"], 1))

  return parameters

In [ ]:
initialize_parameters(nn_architecture, initialization="range_initialization")

# Forward Propogation

In [5]:
# Z = W * X + b
# Here A is output of previous layer

def linear_forward(A_prev, W, b):
  return np.dot(W, A_prev) + b

# apply activation h:  A = h(X) 
def apply_activation(A, activation, alpha=0.01):

  if activation == "sigmoid":
    return activations.sigmoid(A)
  elif activation == "tanh":
    return activations.tanh(A)
  elif activation == "relu":
    return activations.relu(A)
  elif activation == "leaky_relu":
    return activations.leaky_relu(A, alpha)
  elif activation == 'none':
    return A;
  else:
    print(f"ERROR: {activation} activation not supported")
    sys.exit(1)
  
# driver forward propogation
def model_forward(X, parameters, nn_architecture, alpha=0.01):

  forward_cache = {}
  A = X
  num_layers = len(nn_architecture)

  for i in range(1, num_layers):
    A_prev = A
    W = parameters["W" + str(i)]
    b = parametes["b" + str(i)]

    Z = linear_forward(A_prev, W, b)
    activation = nn[i]['activation']
    A = apply_activation(Z, activation, alpha)

    forward_cache['Z' + str(i)] = Z
    forward_cache['A' + str(i - 1)] = A

  return A, forward_cache  

# Backpropogation 

In [7]:
def linear_backward(dz, cache):
  A_prev, W, b = cache
  m = A_prev.shape[1]

  dw = (1 / m) * np.dot(dz, A_prev.T)
  db = (1 / m) * np.sum(dZ, axis=1, keepims=True)
  dA_prev = np.dot(W.T, dZ)

  assert dA_prev.shape == A_prev.shape
  assert dW.shape == W.shape
  assert db.shape == b.shape

  return dA_prev, dw, db      

In [8]:
def apply_activation_backward(dA, cache, activation_fn):
  linear_cache, activation_cache = cache

  if activation_fn == "sigmoid":
    dZ = activations.sigmoid_backward(dA, activation_cache)
    dA_prev, dw, db = linear_backward(dZ, linear_cache)

  elif activation_fn == "tanh":
    dZ = activations.sigmoid_backward(dA, activation_cache)
    dA_prev, dw, db = linear_backward(dZ, linear_cache)

  elif activation_fn == "relu":
    dZ = activations.sigmoid_backward(dA, activation_cache)
    dA_prev, dw, db = linear_backward(dZ, linear_cache)

  elif activation_fn == "leaky_relu":
    dZ = activations.sigmoid_backward(dA, activation_cache)
    dA_prev, dw, db = linear_backward(dZ, linear_cache)
  
  elif activation_fn == "none":
    dZ = dA
    dA_prev, dw, db = linear_backward(dZ, linear_cache)
  
  else:
    print("Activation not available")
    sys.exit(1)

  return dA_prev, dw, db

  


In [4]:
def model_backward(AL, y, caches, nn_architecture):
   
  y = y.reshape(AL.shape)
  L = len(caches)
  grads = {}

  dAL = np.divide(AL - y, np.multiply(AL, 1 - AL))

  grads["dA" + str(L - 1)], grads["dW" + str(L)], grads["db" + str(L)] = \
             apply_activation_backward(dAL, caches[L-1], nn_architecture[L-1]['activation'])

  for l in range(L - 1, 0, -1):
    current_cache = caches[l - 1]
    grads["dA" + str(l - 1)], grads["dW" + str(l)],  \
        grads["db" + str(l)] = apply_activation_backward(
            grads["dA" + str(l)], current_cache, 
            nn_architecture[l]['activation']
        )
  return grads

In [5]:
def update_parameters(parameters, grads, lr):

  L = len(parameters)

  for l in range(1, L + 1):
    parameters["W" + str(l)] = parameters["W" + str(l)] - \
            lr * grads["dW" + str(l)]
    parametets["b" + str(l)] = parameters["b" + str(l)] - \
            lr * grads["db" + str(l)]
  return parameters